# Chapter 11: Testing with Out-of-Sample Data

In [8]:
import sys
import os
sys.path.append('/home/yutanagano/Projects/nnfs')
os.chdir('/home/yutanagano/Projects/nnfs')

In [9]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data
import nnn

nnfs.init()

## Setup

In [10]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)

# Create the necessary layers
dense1 = nnn.layer.Dense(n_inputs=2,n_neurons=64,l2w=5e-4,l2b=5e-4)
relu = nnn.activation.Relu()
dense2 = nnn.layer.Dense(n_inputs=64,n_neurons=3)
activation_loss = nnn.loss.SoftmaxWithCategoricalCrossentropy()

# Create the optimiser
optimiser = nnn.optimiser.Adam(learning_rate=0.01, decay=1e-5)

## Training

In [11]:
# Training loop
for epoch in range(10001):
    # Forward pass
    output = dense1.forward(X)
    output = relu.forward(output)
    output = dense2.forward(output)

    # Calculate the network's current loss
    data_loss = activation_loss.forward(output, y)
    reg_loss = activation_loss.categoricalcrossentropy.regularsiation_loss(dense1) +\
        activation_loss.categoricalcrossentropy.regularsiation_loss(dense2)
    loss = data_loss + reg_loss

    # Calculate accuracy
    predictions = np.argmax(output,axis=1)
    if len(y.shape) == 2: y = np.argmax(y,axis=1)
    accuracy = np.mean(predictions == y)

    # Print accuracy
    if not epoch % 100: print(
        f"epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}, "
        f"(data_loss: {data_loss}, reg_loss: {reg_loss}), "
        f"lr: {optimiser.current_learning_rate}"
    )

    # Backward pass
    activation_loss.backward(activation_loss.outputs, y)
    dense2.backward(activation_loss.dinputs)
    relu.backward(dense2.dinputs)
    dense1.backward(relu.dinputs)

    # Update weights and biases
    optimiser.pre_update_params()
    optimiser.update_params(dense1)
    optimiser.update_params(dense2)
    optimiser.post_update_params()

epoch: 0, acc: 0.360, loss: 1.099, (data_loss: 1.098594307899475, reg_loss: 6.1487173661589625e-06), lr: 0.01
epoch: 100, acc: 0.713, loss: 0.765, (data_loss: 0.7357513308525085, reg_loss: 0.029340030431747436), lr: 0.009990109791306606
epoch: 200, acc: 0.807, loss: 0.595, (data_loss: 0.5536873936653137, reg_loss: 0.040994081497192386), lr: 0.009980139522350523
epoch: 300, acc: 0.833, loss: 0.513, (data_loss: 0.46612802147865295, reg_loss: 0.04675359606742859), lr: 0.009970189134487882
epoch: 400, acc: 0.807, loss: 0.508, (data_loss: 0.4565824270248413, reg_loss: 0.051665717363357545), lr: 0.009960258568312435
epoch: 500, acc: 0.870, loss: 0.433, (data_loss: 0.3764248788356781, reg_loss: 0.05682311868667603), lr: 0.009950347764654375
epoch: 600, acc: 0.887, loss: 0.420, (data_loss: 0.3618479371070862, reg_loss: 0.05859713172912598), lr: 0.009940456664579172
epoch: 700, acc: 0.850, loss: 0.456, (data_loss: 0.3940895199775696, reg_loss: 0.061523983955383305), lr: 0.009930585209386389
epo

## Evaluation

In [12]:
# Create the validation data set
X_test, y_test = spiral_data(samples=100, classes=3)

In [13]:
# Forward pass on validation set
output = dense1.forward(X_test)
output = relu.forward(output)
output = dense2.forward(output)

# Calculate validation loss
loss = activation_loss.forward(output, y_test)

In [14]:
# Calculate accuracy
predictions = np.argmax(output, axis=1)
if len(y_test.shape) == 2:
    y_test = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == y_test)

print(f"validation, acc: {accuracy:.3f}, loss: {loss:.3f}")

validation, acc: 0.813, loss: 0.530
